In [ ]:
import os
import pandas as pd 
import numpy as np
from natsort import natsorted
from natsort import natsort_keygen

pd.set_option('display.max_rows', 10)

from sklearn.metrics import pairwise_distances
import statsmodels.api as sm
import nibabel as nb
import nilearn
from nilearn import datasets
from nilearn.maskers import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure

## <font color='teal'>Preparing the rating data</font>

### <font color='darkgoldenrod'>Dartmouth participants rating data</font>
Compute the absolute difference between each pair of rating participants (who rated the same trait)

In [ ]:
df = pd.read_csv('behavioral_matrix.csv')
phys_df = df[(df.target == 'physical')].copy()
stut_df = df[(df.target == 'stutterer')].copy()

phys_df.sort_values(by=['version', 'subID'], key=natsort_keygen(), inplace=True)
stut_df.sort_values(by=['version', 'subID'], key=natsort_keygen(), inplace=True)

phys_X = phys_df[['bossy','conscientious', 'easygoing', 'humble', 'nosy', 'rebellious']].values
phys_distance_matrix = pairwise_distances(phys_X, metric='euclidean')

stut_X = stut_df[['bossy','conscientious', 'easygoing', 'humble', 'nosy', 'rebellious']].values
stut_distance_matrix = pairwise_distances(stut_X, metric='euclidean')

### <font color='darkgoldenrod'>CloudResearch participants rating data</font>

calculate the average distances among the participants who saw the same version of the same clip for a given movie


In [73]:
def calculate_distances(df, movie, trait, clip):
    #distance among v1 group
    v1_df = df[ (df.movie == movie) & (df.version == 1) & (df.trait == trait) & (df['round'] == clip) ].copy()
    v1_ratings_arr = np.array(v1_df.ratings).reshape(-1,1)
    v1_ratings_distances = pairwise_distances(v1_ratings_arr, metric='euclidean')
    v1_lower_triangle = nilearn.connectome.sym_matrix_to_vec(v1_ratings_distances, discard_diagonal=True)
    v1_rating_avg = v1_lower_triangle.mean()

    #distance among v2 group
    v2_df = df[ (df.movie == movie) & (df.version == 2) & (df.trait == trait) & (df['round'] == clip) ].copy()
    v2_ratings_arr = np.array(v2_df.ratings).reshape(-1,1)
    v2_ratings_distances = pairwise_distances(v2_ratings_arr, metric='euclidean')
    v2_lower_triangle = nilearn.connectome.sym_matrix_to_vec(v2_ratings_distances, discard_diagonal=True)
    v2_rating_avg = v2_lower_triangle.mean()

    #compare across version groups: 
    v1_vs_v2 = pairwise_distances(v1_ratings_arr, v2_ratings_arr, metric='euclidean')
    v1_vs_v2_avg = v1_vs_v2.mean()

    return v1_rating_avg, v2_rating_avg, v1_vs_v2_avg

## <font color='teal'> IS-RSA</font>

We take our parcellated timeseries for each of our clips, with each subject per row, and create a correlation matrix. We take the lower triangle of this correlated matrix and then correlate it with the lower triangle of the behavioral rating data correlation matrix.  

We will do this for each movie separately.

In [74]:
"""
IMPORTANT! Start by selecting the intended monologue type before running:
"""
monologue_type = "IM"
#monologue_type = "NIM"

In [ ]:
segmented_clip_path = '/Volumes/Scraplab/fSEND/inarr_data/'
sublist = [x for x in os.listdir(segmented_clip_path) if 'sub' in x]
complete_cliplist = []

type = "_"+monologue_type

#compile all of the clip files across subjects
for sub in sublist:
    try:
        segment_list = [x for x in os.listdir(segmented_clip_path+sub+os.sep+"segmented_files/") if type in x]

        for f in segment_list:
            complete_cliplist.append(f)
    except:
        continue

movie_list = ['physical', 'stutterer']
traitlist = ['Rebellious', 'Nosy', 'Easygoing', 'Humble', 'Bossy','Conscientious']

im_clips = ['clip-1','clip-3','clip-5','clip-7','clip-9','clip-11','clip-13',
             'clip-15','clip-17','clip-19','clip-21','clip-23']

nim_clips = ['clip-2','clip-4','clip-6','clip-8','clip-10','clip-12',
             'clip-14','clip-16','clip-18','clip-20','clip-22','clip-24'] #we drop the first clip, clip-0!

if monologue_type == "IM":
    working_list = im_clips
elif monologue_type == "NIM":
    working_list = nim_clips

# ONLINE PARTICIPANTS DATA 
cloud_ratings = pd.read_csv("/Users/lindseytepfer/Documents/phd/inarr/online-results/inarr-online-unstacked.csv")
version_matrix = np.loadtxt("version_matrix_ISrsa.csv",delimiter=",", dtype=str, encoding="utf-8-sig")


In [ ]:
schaefer_atlas = datasets.fetch_atlas_schaefer_2018(n_rois=400, yeo_networks=17, resolution_mm=1,
                                                    data_dir=None, base_url=None, resume=True, verbose=1)

masker = NiftiLabelsMasker(
    labels_img=schaefer_atlas.maps,
    strategy='mean',  # Averages voxels in parcel at each TR
    standardize=False  # don't want z-scoring
)

In [ ]:
for movie in movie_list[0:1]:

    movie_cliplist = [x for x in complete_cliplist if movie in x]

    clip_to_behavior_correlation = np.zeros([12,400])

    for cx, clip in enumerate(working_list[1:2]):

        clip_number = int(clip.split("-")[1])
        trait_arr = np.empty([6,3])

        for ix, trait in enumerate(traitlist):
            trait_arr[ix,:] = calculate_distances(cloud_ratings, movie, trait, clip_number)
        
        #get v1, v2, and v1v2 values:
        trait_avg = trait_arr.mean(axis=0)

        #build out clip-specific rating matrix based on participant version matrix
        rating_matrix = np.empty([28,28])

        for r in range(28):
            for c in range(28):
                if version_matrix[r,c] == 'v1v1':
                    rating_matrix[r,c] = trait_avg[0]
                elif version_matrix[r,c] == 'v2v2':
                    rating_matrix[r,c] = trait_avg[1]
                else:
                    rating_matrix[r,c] = trait_avg[2]

        #now resume building the clip's subject-wide ISC matrix
        filtered_clips = [x for x in movie_cliplist if clip+"_" in x]
        
        #this ensures that the subject order matches the distance_matrices orders
        v1_subs = natsorted([x for x in filtered_clips if 'version-1' in x])
        v2_subs = natsorted([x for x in filtered_clips if 'version-2' in x])

        single_clip_matrix_list = []

        for sub in v1_subs:

            sname = sub.split("_")[0]

            clip_segment_nii = nb.load(segmented_clip_path+sname+'/segmented_files/'+sub)

            time_by_parcel = masker.fit_transform(clip_segment_nii) #producess a timepoint x parcel csv, eg (2, 400)
            single_clip_matrix_list.append(time_by_parcel)
        
        for sub in v2_subs:

            sname = sub.split("_")[0]

            clip_segment_nii = nb.load(segmented_clip_path+sname+'/segmented_files/'+sub)

            time_by_parcel = masker.fit_transform(clip_segment_nii) 
            single_clip_matrix_list.append(time_by_parcel)
        
        #build the ISC array
        single_clip_mat_arr = np.array(single_clip_matrix_list)
        
        for p in range(400):
            sub_clip_corr = np.corrcoef(single_clip_mat_arr[:,:,p]) #(28,28)
            ratings_flat, isc_flat = rating_matrix.flatten(), sub_clip_corr.flatten()
            const = sm.add_constant(ratings_flat)
            model = sm.OLS(isc_flat, const)
            results = model.fit()
            clip_to_behavior_correlation[cx,p] = results.params[1] 

### <font color='teal'>Mantel Test (Permutations)</font>

In [ ]:
for movie in movie_list[0:1]:

    movie_cliplist = [x for x in complete_cliplist if movie in x]

    clip_to_version_permutations = np.zeros([12,400,10000])

    for cx, clip in enumerate(working_list[1:2]):

        clip_number = int(clip.split("-")[1])
        trait_arr = np.empty([6,3])

        for ix, trait in enumerate(traitlist):
            trait_arr[ix,:] = calculate_distances(cloud_ratings, movie, trait, clip_number)
        
        #get v1, v2, and v1v2 values:
        trait_avg = trait_arr.mean(axis=0)

        #build out clip-specific rating matrix based on participant version matrix
        rating_matrix = np.empty([28,28])

        for r in range(28):
            for c in range(28):
                if version_matrix[r,c] == 'v1v1':
                    rating_matrix[r,c] = trait_avg[0]
                elif version_matrix[r,c] == 'v2v2':
                    rating_matrix[r,c] = trait_avg[1]
                else:
                    rating_matrix[r,c] = trait_avg[2]

        #now resume building the clip's subject-wide ISC matrix
        filtered_clips = [x for x in movie_cliplist if clip+"_" in x]
        
        #this ensures that the subject order matches the distance_matrices orders
        v1_subs = natsorted([x for x in filtered_clips if 'version-1' in x])
        v2_subs = natsorted([x for x in filtered_clips if 'version-2' in x])

        single_clip_matrix_list = []

        for sub in v1_subs:

            sname = sub.split("_")[0]

            clip_segment_nii = nb.load(segmented_clip_path+sname+'/segmented_files/'+sub)

            time_by_parcel = masker.fit_transform(clip_segment_nii) #producess a timepoint x parcel csv, eg (2, 400)
            single_clip_matrix_list.append(time_by_parcel)
        
        for sub in v2_subs:

            sname = sub.split("_")[0]

            clip_segment_nii = nb.load(segmented_clip_path+sname+'/segmented_files/'+sub)

            time_by_parcel = masker.fit_transform(clip_segment_nii) 
            single_clip_matrix_list.append(time_by_parcel)
        
        #build the ISC array
        single_clip_mat_arr = np.array(single_clip_matrix_list)
        
        for p in range(400):
            np.random.seed(0)
            sub_clip_corr = np.corrcoef(single_clip_mat_arr[:,:,p]) #(28,28)
            
            for perm in range(2):
                random_array = np.random.permutation(28)
                shuffled_matrix = sub_clip_corr[random_array][:, random_array]

                ratings_flat, isc_flat = rating_matrix.flatten(), shuffled_matrix.flatten()
                const = sm.add_constant(ratings_flat)
                model = sm.OLS(isc_flat, const)
                results = model.fit()

                clip_to_version_permutations[cx,p, perm] = results.params[1] 
    
    np.save(movie+'_isRSA_cleaned_permutation_matrix_'+monologue_type, clip_to_version_permutations.mean(axis=0)) #average across clips

# <font color='teal'>Visualize the results</font>

In [ ]:
from nilearn import plotting, datasets, image, surface
import nibabel as nb
from nilearn.plotting import plot_img_on_surf
from nilearn.image import new_img_like

fsaverage = datasets.fetch_surf_fsaverage()

schaefer_atlas = datasets.fetch_atlas_schaefer_2018(n_rois=400, yeo_networks=17, resolution_mm=1,
    data_dir=None, base_url=None, resume=True, verbose=1)

### <font color='darkgoldenrod'>Internal Monologues IS-RSA</font>

In [ ]:
#read in the two different movie permutations and then average across them
phys_im, stut_im = pd.read_csv('physical_clips_by_parcel_IM.csv', index_col=0), pd.read_csv('stutterer_clips_by_parcel_IM.csv', index_col=0)
phys_im_avg, stut_im_avg = phys_im.mean(axis=0).reset_index(drop=True), stut_im.mean(axis=0).reset_index(drop=True)
im_avg = np.array(list((phys_im_avg + stut_im_avg) / 2)) # (400)

phys_perm_im,  stut_perm_im = np.load('/Volumes/Scraplab/lindseytepfer/inarr/physical_isRSA_permutation_matrix_IM.npy'), np.load('/Volumes/Scraplab/lindseytepfer/inarr/stutterer_isRSA_permutation_matrix_IM.npy')
perm_avg_im = np.mean([phys_perm_im, stut_perm_im], axis=0) # (400, 10000)

permuted_vals = np.abs(perm_avg_im) # (400, 10000)
null_dist = permuted_vals.max(axis=0) #shape = (10000)

results_pvals = np.empty((400))

for parcel in range(im_avg.shape[0]): #400, for each parcel
    results_pvals[parcel] = np.mean(null_dist[:] >= abs(im_avg[parcel]))

significant = np.where(results_pvals < 0.01, 1, 0) #(400)

In [ ]:
#create corr image
schaefer_atlas = datasets.fetch_atlas_schaefer_2018(n_rois=400, yeo_networks=17, resolution_mm=1,
    data_dir=None, base_url=None, resume=True, verbose=1)

#need to insert the 0 for proper indexing. 
schaefer_atlas.labels = np.insert(schaefer_atlas.labels, 0, "Background")

atlas = nb.load(schaefer_atlas.maps) # .maps provides the path to the map
atlas_data = atlas.get_fdata()

mapped_data = np.zeros_like(atlas_data)
mapped_data[atlas_data == 0] = np.nan # mark
unique_regions = np.unique(atlas_data)[1:]

for i, region in enumerate(unique_regions):
    mapped_data[atlas_data == region] = im_avg[i]

corr_im_img = nb.Nifti1Image(mapped_data, atlas.affine)
#nb.save(corr_im_img, 'corr_im_map.nii.gz')

In [ ]:
#Create corrected p-value mask:
n_rois = 399

roi_data = image.get_data(atlas)  # (x,y,z) array with ROI labels
significant_mask = np.zeros_like(roi_data, dtype=np.float32)

for roi_label in range(1, n_rois+1):
    if significant[roi_label-1] == 1:  # ROI labels start at 1
        significant_mask[roi_data == roi_label] = 1  # Tag significant voxels

mask_img = new_img_like(atlas, significant_mask)

In [ ]:
#plot Internal Monologue results

plotting.plot_img_on_surf(corr_im_img,
    "fsaverage", inflate=True,
    views=['lateral', 'medial'],  # Hemispheric views to display
    hemispheres=['left', 'right'],  # Both hemispheres
    #threshold=0.05,  # Highlight significant p-values
    cmap='seismic',  # Colormap (e.g., 'viridis', 'coolwarm')
    colorbar=True,  # Show colorbar
    mask_img=mask_img,
    vmin=-1, vmax=1, #removing these options to let it be automatic
)

# fig = plt.gcf()
# fig.savefig('isc_corrected.png', dpi=300, facecolor='w', bbox_inches='tight')
# plt.close()

plotting.show()

### <font color='darkgoldenrod'>Non-internal Monologues IS-RSA</font>

In [ ]:
#read in the two different movie permutations and then average across them
phys_nim, stut_nim = pd.read_csv('physical_clips_by_parcel_NIM.csv',index_col=0), pd.read_csv('stutterer_clips_by_parcel_NIM.csv',index_col=0)
phys_nim_avg, stut_nim_avg = phys_nim.mean(axis=0).reset_index(drop=True), stut_nim.mean(axis=0).reset_index(drop=True)

nim_avg = np.array(list((phys_nim_avg + stut_nim_avg) / 2))
phys_perm_nim,  stut_perm_nim = np.load('/Volumes/Scraplab/lindseytepfer/inarr/physical_isRSA_permutation_matrix_NIM.npy'), np.load('/Volumes/Scraplab/lindseytepfer/inarr/stutterer_isRSA_permutation_matrix_NIM.npy')
perm_avg_nim = np.mean([phys_perm_nim, stut_perm_nim], axis=0) # (400, 10000)

permuted_vals = np.abs(perm_avg_nim) # (400, 10000)
null_dist = permuted_vals.max(axis=0) #shape = (10000)

results_pvals = np.empty((400))

for parcel in range(nim_avg.shape[0]): #400, for each parcel
    results_pvals[parcel] = np.mean(null_dist[:] >= abs(nim_avg[parcel]))

significant = np.where(results_pvals < 0.01, 1, 0) #(400)

In [ ]:
#create corr image
schaefer_atlas = datasets.fetch_atlas_schaefer_2018(n_rois=400, yeo_networks=17, resolution_mm=1,
    data_dir=None, base_url=None, resume=True, verbose=1)

#need to insert the 0 for proper indexing. 
schaefer_atlas.labels = np.insert(schaefer_atlas.labels, 0, "Background")

atlas = nb.load(schaefer_atlas.maps) # .maps provides the path to the map
atlas_data = atlas.get_fdata()

mapped_data = np.zeros_like(atlas_data)
mapped_data[atlas_data == 0] = np.nan # mark
unique_regions = np.unique(atlas_data)[1:]

for i, region in enumerate(unique_regions):
    mapped_data[atlas_data == region] = nim_avg[i]

corr_nim_img = nb.Nifti1Image(mapped_data, atlas.affine)
#nb.save(corr_nim_img, 'corr_nim_map.nii.gz')

In [ ]:
#Create corrected p-value mask:
n_rois = 399

roi_data = image.get_data(atlas)  # (x,y,z) array with ROI labels
significant_mask = np.zeros_like(roi_data, dtype=np.float32)

for roi_label in range(1, n_rois+1):
    if significant[roi_label-1] == 1:  # ROI labels start at 1
        significant_mask[roi_data == roi_label] = 1  # Tag significant voxels

mask_img = new_img_like(atlas, significant_mask)

In [ ]:
#plot Non-internal Monologue results

plotting.plot_img_on_surf(corr_nim_img,
    "fsaverage", inflate=True,
    views=['lateral', 'medial'],  # Hemispheric views to display
    hemispheres=['left', 'right'],  # Both hemispheres
    #threshold=0.05,  # Highlight significant p-values
    cmap='seismic',  # Colormap (e.g., 'viridis', 'coolwarm')
    colorbar=True,  # Show colorbar
    mask_img=mask_img,
    vmin=-1, vmax=1, #removing these options to let it be automatic
)

# fig = plt.gcf()
# fig.savefig('isc_corrected.png', dpi=300, facecolor='w', bbox_inches='tight')
# plt.close()

plotting.show()